In [8]:

import fhirclient.models.binary as bin
import json
import spacy
from spacy import displacy
from spacypdfreader.spacypdfreader import pdf_reader
from spacy import displacy
from pypdf import PdfReader
import pymupdf
import tabula
import base64
import io
import pymupdf4llm
import pathlib

report_list = ['ORU_R01_DLIMS.txt','ORU_R01_R125.1_R0A.txt','ORU_R01_R125.1_RBS.txt','ORU_R01_R125.1_REP.txt','ORU_R01_R125.1_RR8.txt','ORU_R01_R125.1_RX1.txt','ORU_R01_R125.1_SG9.txt','ORU_R01_R125.1_ZT001.txt','ORU_R01_R125.1_7A3.txt','ORU_R01_R125.1_RPY.txt','ORU_R01_R125.1_RXK.txt', 'SHIRE_ORU_R01_RM3.txt', 'WALES_ORU_R01_TX.txt']

# For testing

report_list = ['ORU_R01_R125.1_R0A.txt','ORU_R01_DLIMS.txt']


pdfReaderEnabled = False
pdfPymupdfEnabled = False
pdfpymupdf4llmEnabled = True
pdfTableEnabled = False
spacyEnabled = True

for file in report_list:
    with open('Output/FHIR/R01/' + file+ '.json', 'rb') as g:
        rawJson = g.read()
        jsonBundle = json.loads(rawJson)

        #bundle = b.Bundle(jsonBundle)
        for entry in jsonBundle['entry']:
            #print(entry['resource']['resourceType'])
            if entry['resource']['resourceType'] == 'Binary':
                binary = bin.Binary(entry['resource'])
                encoded = binary.data
                #print(encoded)
                decode = base64.b64decode(encoded)
                pdf_stream = io.BytesIO(decode)
                if pdfpymupdf4llmEnabled:
                    doc = pymupdf.open("pdf", pdf_stream)
                    md_text = pymupdf4llm.to_markdown(doc)
                    #print(md_text)
                    pathlib.Path('Output/Markdown/R01/' + file+ '.md').write_bytes(md_text.encode())
                if pdfPymupdfEnabled:
                    # https://pymupdf.readthedocs.io/en/latest/the-basics.html

                    # Initial Tests indicate this is the best method to use.

                    doc = pymupdf.open("pdf", pdf_stream)
                    print('Pages: '+ str(len(doc)))
                    print('============ pymupdf ' + file + ' ==============')
                    for page in doc: # iterate the document pages
                        text = page.get_text().encode("utf8")
                        #print(text)
                if pdfReaderEnabled:
                    pdf = PdfReader(pdf_stream)
                    print('Pages: '+ str(len(pdf.pages)))
                    print('============ PDF Reader ' + file + ' ==============')
                    for page in pdf.pages:
                        text = page.extract_text()
                        # print(text)
                if pdfTableEnabled:
                    tables = tabula.read_pdf(pdf_stream
                                             , pages='all', multiple_tables=True)
                    print('============ Tabula ' + file + ' ==============')
                    for table in tables:
                        tab1 = table
                        #print(table)
                with open('Output/PDF/R01/' + file+ '.pdf', 'wb') as pdf:
                    pdf.write(decode)


pdf_list = ['R24-0WH7-1.pdf','R24-1A4W-1.pdf']

for file in pdf_list:
    with open('Output/PDF/R01/' + file, 'rb') as g:
        pdf_stream = g.read()
        if pdfpymupdf4llmEnabled:
            doc = pymupdf.open("pdf", pdf_stream)
            md_text = pymupdf4llm.to_markdown(doc)
            # print(md_text)
            pathlib.Path('Output/Markdown/R01/' + file+ '.md').write_bytes(md_text.encode())
        if spacyEnabled:
            print('============ Spacy ' + file + ' ==============')
            nlp = spacy.load('en_core_web_sm')
            # TODO use pdf_stream instead
            doc = pdf_reader('Output/PDF/R01/' + file, nlp)
            for ent in doc.ents:
                print(ent.text, ent.label_)
            displacy.render(doc, style='ent')

pdf_list = ['genomic-medicine-device-test-order-form-cancer-v1.22.pdf']

for file in pdf_list:
    with open('Output/PDF/O21/' + file, 'rb') as g:
        pdf_stream = g.read()
        if pdfpymupdf4llmEnabled:
            doc = pymupdf.open("pdf", pdf_stream)
            md_text = pymupdf4llm.to_markdown(doc)
            # print(md_text)
            pathlib.Path('Output/Markdown/O21/' + file+ '.md').write_bytes(md_text.encode())

